In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail").to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [2]:
device

'cuda'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/teamspace/studios/this_studio/NLP_project/openai_cnnanddm_final.csv', sep=';')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6312 entries, 0 to 6311
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    6312 non-null   int64 
 1   user_id       6312 non-null   object
 2   user_profile  6312 non-null   object
 3   doc_id        6312 non-null   object
 4   post/article  6312 non-null   object
 5   summary_text  6291 non-null   object
 6   confidence    6312 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 345.3+ KB


In [6]:
df = df.dropna(subset=['summary_text'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6291 entries, 0 to 6311
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    6291 non-null   int64 
 1   user_id       6291 non-null   object
 2   user_profile  6291 non-null   object
 3   doc_id        6291 non-null   object
 4   post/article  6291 non-null   object
 5   summary_text  6291 non-null   object
 6   confidence    6291 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 393.2+ KB


In [8]:
df['doc_id'].value_counts()

doc_id
DC221    77
DC435    55
DC141    44
DC120    44
DC490    44
         ..
DC51      2
DC545     1
DC285     1
DC529     1
DC356     1
Name: count, Length: 639, dtype: int64

In [9]:
data = df[['doc_id', 'post/article']]

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6291 entries, 0 to 6311
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   doc_id        6291 non-null   object
 1   post/article  6291 non-null   object
dtypes: object(2)
memory usage: 147.4+ KB


In [11]:
mapp = {}

for idx , row in data.iterrows():
    doc_id = row['doc_id']
    text = row['post/article']

    if doc_id not in mapp:
        mapp[doc_id] = text

In [12]:
len(mapp)

639

In [13]:
ids = list(mapp.keys())
len(ids)

639

In [14]:
unique_texts = list(mapp.values())
len(unique_texts)

639

In [24]:
batch_size = 5

# Function to perform inference on a batch of texts
def translate_batch(texts):
    batch = tokenizer(texts, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_texts

In [16]:
results = []

In [25]:
src_texts = unique_texts
len(src_texts)

639

In [26]:
for i in range(0, len(src_texts), batch_size):
    batch_texts = src_texts[i:i + batch_size]
    translated_texts = translate_batch(batch_texts)
    results.extend(translated_texts)


In [34]:
len(results[0])

169

In [30]:
gen_summaries = results

In [31]:
mapp = dict(zip(ids, gen_summaries))

In [32]:
len(mapp)

639

In [33]:
len(mapp['DC0'])

169

In [35]:
df['pegasus_model_summary'] = df['doc_id'].map(mapp)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6291 entries, 0 to 6311
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             6291 non-null   int64 
 1   user_id                6291 non-null   object
 2   user_profile           6291 non-null   object
 3   doc_id                 6291 non-null   object
 4   post/article           6291 non-null   object
 5   summary_text           6291 non-null   object
 6   confidence             6291 non-null   int64 
 7   pegasus_model_summary  6291 non-null   object
dtypes: int64(2), object(6)
memory usage: 442.3+ KB


In [38]:
len(df.iloc[0]['post/article']), len(df.iloc[0]['pegasus_model_summary'])

(2104, 169)

In [39]:
df.to_csv('openai_cnndm_final_pegasus.csv', sep=';')